In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, accuracy_score


In [ ]:
# Load and Prepare Data (Same as before for fair comparison)
print("Loading MNIST data...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

X = ((X / 255.) - .5) * 2

# Convert labels to integers
y = y.astype(int)

Loading MNIST data...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=123, stratify=y
)

# One-hot encode labels for Keras training and AUC calculation
enc = OneHotEncoder()
y_train_onehot = enc.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_onehot = enc.transform(y_test.reshape(-1, 1)).toarray()

In [ ]:
model = tf.keras.models.Sequential([
    # Hidden Layer 1
    tf.keras.layers.Dense(500, activation='sigmoid', input_shape=(784,)),

    # Hidden Layer 2
    tf.keras.layers.Dense(500, activation='sigmoid'),

    # Output Layer: 'softmax'
    tf.keras.layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# 4. Compile
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['accuracy'])


In [ ]:
# 5. Train
print("Training Keras model...")
history = model.fit(X_train, y_train_onehot,
                    epochs=50,
                    batch_size=100,
                    validation_split=0.1,
                    verbose=1)

Training Keras model...
Epoch 1/50
441/441 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.1576 - loss: 0.0914 - val_accuracy: 0.1682 - val_loss: 0.0891
Epoch 2/50
441/441 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.2775 - loss: 0.0888 - val_accuracy: 0.1982 - val_loss: 0.0881
Epoch 3/50
441/441 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.2949 - loss: 0.0877 - val_accuracy: 0.3333 - val_loss: 0.0869
Epoch 4/50
441/441 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.3523 - loss: 0.0865 - val_accuracy: 0.2584 - val_loss: 0.0854
Epoch 5/50
441/441 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.3272 - loss: 0.0848 - val_accuracy: 0.3508 - val_loss: 0.0832
Epoch 6/50
441/441 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.3516 - loss: 0.0824 - val_accuracy: 0.4133 - val_loss: 0.0803
Epoch 7/50
441/441 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.4026 - loss: 0.0793 - val_accuracy: 0.4292 - val_loss: 0.0767
Epoch 8/50
441/441 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4577 

In [ ]:
# 6. Evaluate
print("\nEvaluating performance...")
y_pred_proba = model.predict(X_test)
y_pred_class = np.argmax(y_pred_proba, axis=1)

acc = accuracy_score(y_test, y_pred_class)
macro_auc = roc_auc_score(y_test_onehot, y_pred_proba, average='macro', multi_class='ovr')

print(f"Test Accuracy: {acc*100:.2f}%")
print(f"Macro AUC: {macro_auc*100:.2f}")


Evaluating performance...
657/657 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
Test Accuracy: 90.17%
Macro AUC: 98.99
